Installing libraries

Goal: create a multi agent LLM using LangGraph that does the activities listed below. Use the steps and workflow mentioned in the attached LangGraph reference project to produce the code. 

Activities that LangGraph multi agent will do
1. Takes dynamic user input on target company. User either provides company name or uploads  document that has company info. 
2. Based on company name, does a web search for company description, names of top 5 C level execs, lists the company's top 3 priorities, and identifies top trends for the industry company operates in
3. Checks on LinkedIn if the execs are still at the company (current company = company name inputted). If exec is not at company anymore, then exclude that exec from the list
4. Takes input of your company's URL. Can also take in as an input any existing material you have for your company. 
5. Drafts an initial sales pitch email that can be sent to each of the identified execs

Tools used: LangGraph, Tavily, OpenAI, Qdrant, Pymupdf, 

These are all the agents that will get built:
Company Research Alpha
Asks the user input on target company. User either provides company name or uploads  document that has company info. 
Initial documents to be created include company name, company description, company execs, company priorities, and industry priorities
Company Description Team
Initial document created to contain company description. 
A. Research Team
1. Company Research Supervisor Agent
2. Company Web Search Agent - does web search to get the company description
3. Company RAG agent

B. Writing Team
1. Company Description Writing Supervisor: takes the document prepared by Company Research Team
2. Fact checker - checks to make sure company description is correct
3. Copy editor - ensures grammar, spelling, and tone is correct

Company Execs Team
A. Research Team
1. Research team supervisor: takes the company name and company description document and sends initial query to web search agent. 
2. Web Search Agent - does web search to get the top 5 execs. Suggested titles to include CEO, CFO, COO, CTO, CPO, CRO but close matches are allowed. Pull the full name (first name and last name) and updates the document.
3. Company Execs RAG agent - document gets updated
4. LinkedIn Crosscheck: Supervisor sends updated list (document) to LinkedIn cross check agent. This agent searches LinkedIn to ensure current company for exec = company name inputted by user. 
B. Writing Team
1. Writing team supervisor
2. Editor agent: exclude any names that are a not a match from LinkedIn cross check agent.  
3. Publisher agent: does the final publishing of the names and titles of company execs.

Company Priorities
A. Research Team
1. Research team supervisor: takes the company name, company description document, and company exec list document. Sends initial query to web search agent. 
2. Web search agent: searches the web to identify company's top 3 priorities. Updates document
3. Company Top Priorities RAG agent - document gets updated
B. Writing Team
1. Company Priorities Writing Supervisor: takes the document prepared by Company Priorities Research Team
2. Fact checker - checks to make sure company Priorities are correct
3. Copy editor - ensures grammar, spelling, and tone is correct

Industry trends team
A. Research Team
1. Research team supervisor: takes the company name, company description document, and company exec list, and company priorities documents. Sends initial query to web search agent. 
2. Web search agent industry: searches the web to identify the industry that the company operates in. Updates document. 
3. Web search agent industry priorities: searches the web to identify the top industry trends and challenges. Updates document. 
4. Industry Top Priorities RAG agent - document gets updated
B. Writing Team
1. Industry Priorities Writing Supervisor: takes the document prepared by Industry Priorities Research Team
2. Fact checker - checks to make sure industry Priorities are correct
3. Copy editor - ensures grammar, spelling, and tone is correct

Sales Pitch Team
Sales Collateral Supervisor Agent: asks the user input on user's company. User either provides company name or uploads document that has company info. 
A. Value Proposition Research Team
1. Research team supervisor: takes documents and info on user's company. Sends initial query to web search agent. 
2. Web search agent industry: searches the web to outline the value proposition of user company. Updates document. 
3. Industry Top Priorities RAG agent - document gets updated
B. Writing Team
1. Value Proposition Writing Supervisor: takes the document prepared by Value Proposition Research Team
2. Fact checker - checks to make sure value proposition correct
3. Copy editor - ensures grammar, spelling, and tone is correct

Collateral Creation Team
1. Sets all the documents as context in a vector database
2. Sends a query to the database to prepare a sales pitch based on value proposition mapping to company description, company priorities, and industry priorities. 
3. Fact checker - checks to make sure sales collateral is correct
4. Copy editor - ensures grammar, spelling, and tone is correct

In [1]:
pip install -qU langgraph langchain langchain_openai langchain_community qdrant-client PyPDF2 beautifulsoup4 requests

Note: you may need to restart the kernel to use updated packages.


Setting up imports

In [2]:
import os
import logging
import json
import getpass
from typing import List, Dict, Any, Union, Annotated, Optional
from pathlib import Path

from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import BaseTool, tool
from langgraph.graph import StateGraph, END
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import Qdrant
from qdrant_client import QdrantClient
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent
from langchain.prompts import StringPromptTemplate
from langchain.chains import LLMChain
from langchain.schema import AgentAction, AgentFinish

# Set up logging
logging.basicConfig(level=logging.INFO)

# Set up API keys
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API Key:")

Create a working directory

In [3]:
# Create a working directory
WORKING_DIRECTORY = Path("./content/data")
WORKING_DIRECTORY.mkdir(parents=True, exist_ok=True)

Initialize Qdrant Client and OpenAI Embeddings

In [4]:
# Initialize Qdrant client and OpenAI embeddings
qdrant_client = QdrantClient(":memory:")
embeddings = OpenAIEmbeddings(openai_api_key="OPENAI_API_KEY")
 

State and Helper functions

In [5]:
# Define the State
class State(Dict[str, Any]):
    messages: List[Union[HumanMessage, AIMessage]]
    next: str
    documents: Dict[str, str]

# Helper functions
def get_last_message(state: State) -> str:
    return state["messages"][-1].content

Setting up Vector Store

In [6]:
def initialize_vector_store():
    qdrant_client.recreate_collection(
        collection_name="sales_pitch_data",
        vectors_config={"size": 1536, "distance": "Cosine"},
    )

def add_to_vector_store(text: str, metadata: Dict[str, Any]):
    vector = embeddings.embed_query(text)
    qdrant_client.upsert(
        collection_name="sales_pitch_data",
        points=[{"id": str(len(vector)), "vector": vector, "payload": metadata}],
    )

def query_vector_store(query: str, limit: int = 5) -> List[Dict[str, Any]]:
    query_vector = embeddings.embed_query(query)
    search_result = qdrant_client.search(
        collection_name="sales_pitch_data",
        query_vector=query_vector,
        limit=limit,
    )
    return [hit.payload for hit in search_result]

Tools

In [23]:
# Tools
tavily_tool = TavilySearchResults(max_results=5)

import requests
from bs4 import BeautifulSoup
import time
import random
from typing import Optional

@tool
def linkedin_check(name: str, company: str) -> bool:
    """
    Check if a person works at a specific company on LinkedIn.
    
    Args:
    name (str): The name of the person to check.
    company (str): The name of the company to check against.
    
    Returns:
    bool: True if the person is found to work at the company, False otherwise.
    """
    # ... (rest of the function remains the same)

@tool
def read_document(file_name: str, start: Optional[int] = None, end: Optional[int] = None) -> str:
    """
    Read the content of a document within the specified line range.
    
    Args:
    file_name (str): The name of the file to read.
    start (Optional[int]): The starting line number (inclusive). If None, starts from the beginning.
    end (Optional[int]): The ending line number (exclusive). If None, reads to the end.
    
    Returns:
    str: The content of the document within the specified range.
    """
    with (WORKING_DIRECTORY / file_name).open("r") as file:
        lines = file.readlines()
    return "".join(lines[start:end])

@tool
def write_document(content: str, file_name: str) -> str:
    """
    Write content to a document file.
    
    Args:
    content (str): The content to write to the file.
    file_name (str): The name of the file to write to.
    
    Returns:
    str: A message confirming where the document was saved.
    """
    file_path = WORKING_DIRECTORY / file_name
    with file_path.open("w") as file:
        file.write(content)
    return f"Document saved to {file_path}"

@tool
def query_vector_db(query: str) -> str:
    """
    Query the vector database with the given query string.
    
    Args:
    query (str): The query string to search for in the vector database.
    
    Returns:
    str: A string representation of the query results.
    """
    results = query_vector_store(query)
    return "\n".join([f"{item['type']}: {item['content']}" for item in results])

Agent Creation

In [24]:
from langchain.agents import AgentOutputParser
from langchain.schema import AgentAction, AgentFinish
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent
from langchain.prompts import StringPromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from typing import List, Union
import re

class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        
        # If no specific action is mentioned, treat the entire output as the final answer
        return AgentFinish(
            return_values={"output": llm_output.strip()},
            log=llm_output,
        )
        
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

import os
from dotenv import load_dotenv

# Load environment variables from .env file (if you're using one)
load_dotenv()

from langchain_openai import ChatOpenAI

def create_agent(name: str, tools: List[Tool], prompt: str):
    llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
    
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", f"You are {name}. {prompt}"),
        ("human", "Task: {input}"),
        ("human", "Previous steps: {agent_scratchpad}")
    ])
    
    output_parser = CustomOutputParser()
    
    agent = LLMSingleActionAgent(
        llm_chain=LLMChain(llm=llm, prompt=prompt_template),
        output_parser=output_parser,
        allowed_tools=[tool.name for tool in tools],
        stop=["\nObservation:"],
    )
    
    return AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

Agent Nodes

In [25]:
# Define agent nodes
def create_agent_node(name: str, tools: List[Tool], prompt: str):
    agent = create_agent(name, tools, prompt)
    
    def node_function(state: State):
        try:
            last_message = state["messages"][-1].content if state["messages"] else ""
            agent_input = {
                "input": last_message,
                "agent_scratchpad": "",
            }
            agent_output = agent.invoke(agent_input)
            
            # Write output to file
            output_file = f"{name}.txt"
            with open(WORKING_DIRECTORY / output_file, 'w') as f:
                f.write(agent_output['output'])
            
            next_step = node_order[node_order.index(name) + 1] if name in node_order else "END"
            
            return {
                "messages": state["messages"] + [AIMessage(content=agent_output['output'])],
                "next": next_step,
                "documents": {**state.get("documents", {}), name: output_file},
            }
        except Exception as e:
            logging.error(f"Error in {name}: {str(e)}")
            return {
                "messages": state["messages"] + [AIMessage(content=f"Error in {name}: {str(e)}")],
                "next": "END",
                "documents": {**state.get("documents", {}), name: f"Error: {str(e)}"},
            }
    
    return node_function

Defining Agent Nodes

In [26]:
# Define all agent nodes
node_definitions = [
    ("CompanyResearchAlpha", [tavily_tool, read_document, write_document, query_vector_db], 
     "You are a company research specialist. Your task is to gather initial information about a target company."),
    ("CompanyDescriptionResearchSupervisor", [tavily_tool, read_document, write_document, query_vector_db], 
     "You are a research supervisor. Your task is to oversee the company description research process."),
    ("CompanyWebSearchAgent", [tavily_tool, write_document], 
     "You are a web search agent. Your task is to search the web for the company description."),
    ("CompanyRAGAgent", [read_document, write_document, query_vector_db],
     "You are a RAG agent. Your task is to process and store the company description."),
    ("CompanyDescriptionWritingSupervisor", [read_document, write_document],
     "You are a writing supervisor. Your task is to oversee the company description writing process."),
    ("CompanyDescriptionFactChecker", [tavily_tool, read_document, write_document],
     "You are a fact checker. Your task is to verify the accuracy of the company description."),
    ("CompanyDescriptionCopyEditor", [read_document, write_document],
     "You are a copy editor. Your task is to ensure the company description has correct grammar, spelling, and tone."),
    ("CompanyExecsResearchSupervisor", [tavily_tool, read_document, write_document],
     "You are a research supervisor. Your task is to oversee the company executives research process."),
    ("CompanyExecsWebSearchAgent", [tavily_tool, write_document],
     "You are a web search agent. Your task is to search for the top 5 executives of the company."),
    ("CompanyExecsRAGAgent", [read_document, write_document, query_vector_db],
     "You are a RAG agent. Your task is to process and store the company executives information."),
    ("LinkedInCrosscheckAgent", [linkedin_check, read_document, write_document],
     "You are a LinkedIn verification agent. Your task is to verify if the executives are currently working at the company."),
    ("CompanyExecsWritingSupervisor", [read_document, write_document],
     "You are a writing supervisor. Your task is to oversee the company executives writing process."),
    ("CompanyExecsEditor", [read_document, write_document],
     "You are an editor. Your task is to finalize the list of company executives, excluding any that are not currently with the company."),
    ("CompanyExecsPublisher", [read_document, write_document],
     "You are a publisher. Your task is to publish the final list of company executives."),
    ("CompanyPrioritiesResearchSupervisor", [tavily_tool, read_document, write_document],
     "You are a research supervisor. Your task is to oversee the company priorities research process."),
    ("CompanyPrioritiesWebSearchAgent", [tavily_tool, write_document],
     "You are a web search agent. Your task is to search for the company's top 3 priorities."),
    ("CompanyPrioritiesRAGAgent", [read_document, write_document, query_vector_db],
     "You are a RAG agent. Your task is to process and store the company priorities information."),
    ("CompanyPrioritiesWritingSupervisor", [read_document, write_document],
     "You are a writing supervisor. Your task is to oversee the company priorities writing process."),
    ("CompanyPrioritiesFactChecker", [tavily_tool, read_document, write_document],
     "You are a fact checker. Your task is to verify the accuracy of the company priorities."),
    ("CompanyPrioritiesCopyEditor", [read_document, write_document],
     "You are a copy editor. Your task is to ensure the company priorities have correct grammar, spelling, and tone."),
    ("IndustryTrendsResearchSupervisor", [tavily_tool, read_document, write_document],
     "You are a research supervisor. Your task is to oversee the industry trends research process."),
    ("IndustryTrendsWebSearchAgent", [tavily_tool, write_document],
     "You are a web search agent. Your task is to search for the industry trends and challenges."),
    ("IndustryTrendsRAGAgent", [read_document, write_document, query_vector_db],
     "You are a RAG agent. Your task is to process and store the industry trends information."),
    ("IndustryTrendsWritingSupervisor", [read_document, write_document],
     "You are a writing supervisor. Your task is to oversee the industry trends writing process."),
    ("IndustryTrendsFactChecker", [tavily_tool, read_document, write_document],
     "You are a fact checker. Your task is to verify the accuracy of the industry trends."),
    ("IndustryTrendsCopyEditor", [read_document, write_document],
     "You are a copy editor. Your task is to ensure the industry trends have correct grammar, spelling, and tone."),
    ("SalesCollateralSupervisor", [read_document, write_document],
     "You are a sales collateral supervisor. Your task is to oversee the creation of the sales pitch."),
    ("ValuePropositionResearchSupervisor", [tavily_tool, read_document, write_document],
     "You are a research supervisor. Your task is to oversee the value proposition research process."),
    ("ValuePropositionWebSearchAgent", [tavily_tool, write_document],
     "You are a web search agent. Your task is to search for the value proposition of the user's company."),
    ("ValuePropositionRAGAgent", [read_document, write_document, query_vector_db],
     "You are a RAG agent. Your task is to process and store the value proposition information."),
    ("ValuePropositionWritingSupervisor", [read_document, write_document],
     "You are a writing supervisor. Your task is to oversee the value proposition writing process."),
    ("ValuePropositionFactChecker", [tavily_tool, read_document, write_document],
     "You are a fact checker. Your task is to verify the accuracy of the value proposition."),
    ("ValuePropositionCopyEditor", [read_document, write_document],
     "You are a copy editor. Your task is to ensure the value proposition has correct grammar, spelling, and tone."),
    ("SalesPitchCreator", [read_document, write_document, query_vector_db],
     "You are a sales pitch creator. Your task is to create a compelling sales pitch based on all the gathered information."),
    ("SalesPitchFactChecker", [tavily_tool, read_document, write_document],
     "You are a fact checker. Your task is to verify the accuracy of the sales pitch."),
    ("SalesPitchCopyEditor", [read_document, write_document],
     "You are a copy editor. Your task is to ensure the sales pitch has correct grammar, spelling, and tone.")
]

# Create nodes
nodes = {name: create_agent_node(name, tools, prompt) for name, tools, prompt in node_definitions}

# Define a dictionary mapping node names to their corresponding functions
node_functions = {name: nodes[name] for name, _, _ in node_definitions}

Graph Creation

In [27]:
# Define the node order (this should match the order in which you want the nodes to be executed)
node_order = [name for name, _, _ in node_definitions]
# Create the graph
graph = StateGraph(State)

# Add nodes to the graph
for node in node_order:
    graph.add_node(node, node_functions[node])

# Add edges between nodes
for i in range(len(node_order) - 1):
    current_node = node_order[i]
    next_node = node_order[i + 1]
    graph.add_edge(current_node, next_node)

# Add conditional edges for each node
for node in node_order:
    graph.add_conditional_edges(
        node,
        lambda x: x["next"],
        {next_node: next_node for next_node in node_order[node_order.index(node)+1:]} | {"END": END}
    )

# Set the entry point
graph.set_entry_point("CompanyResearchAlpha")

# Compile the graph
workflow = graph.compile()


Main Chain and Execution

In [29]:
# Main execution function
def run_sales_pitch_generator(target_company: str, user_company_url: str):
    initialize_vector_store()
    
    initial_message = f"Generate a sales pitch for {target_company}. Our company website is {user_company_url}."
    
    result = workflow.invoke({
        "messages": [HumanMessage(content=initial_message)],
        "next": "CompanyResearchAlpha",
        "documents": {}
    }, config={"recursion_limit": 50})
    
    final_pitch = compile_final_sales_pitch(result["documents"], target_company)
    
    # Save the final pitch
    with open(WORKING_DIRECTORY / "final_sales_pitch.txt", 'w') as f:
        f.write(final_pitch)
    
    return result["documents"]

def compile_final_sales_pitch(documents: Dict[str, str], target_company: str) -> str:
    pitch = f"Final Sales Pitch for {target_company}\n\n"
    
    sections = [
        ("CompanyDescriptionCopyEditor", "Company Description"),
        ("CompanyExecsPublisher", "Company Executives"),
        ("CompanyPrioritiesCopyEditor", "Company Priorities"),
        ("IndustryTrendsCopyEditor", "Industry Trends"),
        ("ValuePropositionCopyEditor", "Value Proposition"),
        ("SalesPitchCopyEditor", "Sales Pitch")
    ]
    
    for node, section_title in sections:
        file_path = WORKING_DIRECTORY / documents.get(node, "")
        if file_path.exists():
            with open(file_path, 'r') as f:
                content = f.read()
            pitch += f"{section_title}:\n{content[:500]}...\n\n"
        else:
            pitch += f"{section_title}: Information not available.\n\n"
    
    return pitch

if __name__ == "__main__":
    target_company = input("Enter the name of the company you want to prospect: ").strip()
    user_company_url = input("Enter your company's URL: ").strip()
    
    logging.info(f"Generating sales pitch for {target_company}...")
    documents = run_sales_pitch_generator(target_company, user_company_url)
    
    for doc_name, file_name in documents.items():
        print(f"\n{doc_name}:")
        file_path = WORKING_DIRECTORY / file_name
        if file_path.exists():
            with open(file_path, 'r') as f:
                print(f.read())
        else:
            print("Document not available")
    
    print("\nFinal Sales Pitch:")
    with open(WORKING_DIRECTORY / "final_sales_pitch.txt", 'r') as f:
        print(f.read())

INFO:root:Generating sales pitch for Amazon...
/var/folders/jr/_qkyxp313390z32jmym7j9sm0000gp/T/ipykernel_96443/2203980903.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(




> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


As a company research specialist, I will first gather initial information about Amazon before creating a sales pitch. Let's start by looking into key details about Amazon such as its history, products and services, market position, financial performance, and any recent news or developments. I will provide you with a comprehensive overview of Amazon to help you craft an effective sales pitch. Let's begin the research process.

> Finished chain.


> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

1. Target Audience: Young professionals aged 25-35 seeking convenient and healthy meal options.
2. Unique Selling Points: Customizable meal plans, fresh locally sourced ingredients, user-friendly mobile app.
3. Value Proposition: Time-saving convenience, health benefits, personalized options for busy young professionals.

> Finished chain.


> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Task: 1. Target Audience: Young professionals aged 25-35 seeking convenient and healthy meal options.
2. Unique Selling Points: Customizable meal plans, fresh locally sourced ingredients, user-friendly mobile app.
3. Value Proposition: Time-saving convenience, health benefits, personalized options for busy young professionals.

Sales Pitch: "Our meal delivery service is perfect for busy young professionals aged 25-35 who want convenient and healthy meal options. With customizable meal plans, fresh locally sourced ingredients, and a user-friendly mobile app, we offer time-saving convenience, health benefits, and personalized options to fit your busy lifestyle."

Fact Checker: I will now verify the accuracy of the sales pitch.

> Finished chain.


> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
ERROR:root:Error in SalesPitchCopyEditor: list index out of range
INFO:root:Final sales pitch saved to content/data/final_sales_pitch.txt


Sales Pitch: "Our meal delivery service is perfect for busy young professionals aged 25-35 who want convenient and healthy meal options. With customizable meal plans, fresh locally sourced ingredients, and a user-friendly mobile app, we offer time-saving convenience, health benefits, and personalized options to fit your busy lifestyle."

> Finished chain.

Final Sales Pitch:
Final Sales Pitch for Amazon

Company Overview:
As a company research specialist, I will first gather initial information about Amazon before creating a sales pitch. Let's start by looking into key details about Amazon such as its history, products and services, market position, financial performance, and any recent news or developments. I will provide you with a comprehensive overview of Amazon to help you craft an effective sales pitch. Let's begin the research process....

Company Description:
Our company, XYZ Inc., is a leading provider of innovative technology solutions for businesses worldwide. With a team of